# Preprocess the data

changes I've made:
had to implement a yaml file and extract the data neccessary
the file structure for both the jpeg and the txt were wrong so those had to be changed
the naming for both the jpegs and the txt were wrong

Create the dir names

In [2]:
import os
from scipy.io import loadmat

data_folder_dir = "C:/Users/dasul/Downloads/data_and_labels" # this is just the name of the folder of where all the downloaded data is, jsut enter where its stored, and remeber to use forward slashes
converted_data_dir = os.path.join(data_folder_dir, 'processed_data')
converted_images_dir = os.path.join(converted_data_dir, 'images')
converted_labels_dir = os.path.join(converted_data_dir, 'labels')
converted_images_train_dir = os.path.join(converted_images_dir, 'train')
converted_images_test_dir = os.path.join(converted_images_dir, 'test')
converted_labels_train_dir = os.path.join(converted_labels_dir, 'train')
converted_labels_test_dir = os.path.join(converted_labels_dir, 'test')
os.makedirs(converted_data_dir, exist_ok=True)
os.makedirs(converted_images_dir, exist_ok=True)
os.makedirs(converted_labels_dir, exist_ok=True)
os.makedirs(converted_images_train_dir, exist_ok=True)
os.makedirs(converted_images_test_dir, exist_ok=True)
os.makedirs(converted_labels_train_dir, exist_ok=True)
os.makedirs(converted_labels_test_dir, exist_ok=True)

Create the Seq files

In [7]:
import cv2
import os
from glob import glob
from tqdm import tqdm

# base path = data_foler_dir
base_path = data_folder_dir
train_seq_files = glob(os.path.join(base_path, "Train", "set*", "set*", "V*.seq"))
test_seq_files = glob(os.path.join(base_path, "Test", "set*", "set*", "V*.seq"))

# img num will just be used to name the file, could potential cause issues, if the files are not processed in the same order ie vbb and seq
def extract_frames_from_seq(seq_file, output_dir, imgnum):
    #os.makedirs(output_dir, exist_ok=True)
    # i commented out the previous file becuase i already created teh file
    cap = cv2.VideoCapture(seq_file)
    count = 0
    base_name = os.path.splitext(os.path.basename(seq_file))[0]

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # *********** Filtering invalid/small frames ***********
        if frame is None or frame.shape[0] < 100 or frame.shape[1] < 100:
            count += 1
            continue

        # *********** Resize image to YOLO-compatible size ***********
        frame = cv2.resize(frame, (640, 480))

        #filename = os.path.join(output_dir, f"img{imgnum}.jpg")
        filename = os.path.join(output_dir, f"{base_name}_{count:05d}.jpg")
        cv2.imwrite(filename, frame)
        count += 1
        imgnum += 1

    cap.release()

imgnum = 0
# Extract train
for seq_file in tqdm(train_seq_files, desc="Extracting train frames"):
    extract_frames_from_seq(seq_file, os.path.join(base_path, converted_images_train_dir), imgnum)

# Extract tests
for seq_file in tqdm(test_seq_files, desc="Extracting test frames"):
    extract_frames_from_seq(seq_file, os.path.join(base_path, converted_images_test_dir), imgnum)

Extracting test frames: 100%|██████████| 66/66 [29:36<00:00, 26.92s/it]


Convert VBB to Txt

In [8]:
def convert_vbb_to_yolo(vbb_path, output_dir, setnum, vbbnum, image_width=640, image_height=480):
    print(f"Reading .vbb file: {vbb_path}")
    vbb = loadmat(vbb_path)
    obj_lists = vbb['A'][0][0][1][0]  # objLists
    n_frames = vbb['A'][0][0][0][0][0]  # nFrame


    abs_output_dir = os.path.abspath(output_dir)
    os.makedirs(abs_output_dir, exist_ok=True)


    for frame_idx in range(n_frames):
        if obj_lists[frame_idx].size == 0:
           objs = []
        else:
           objs = obj_lists[frame_idx][0]


        label_lines = []


        for obj in objs:
            pos = obj[1][0]  # [x, y, w, h]
            x, y, w, h = pos


            # Convert to YOLO format (center x/y, normalized)
            x_center = (x + w / 2) / image_width
            y_center = (y + h / 2) / image_height
            w_norm = w / image_width
            h_norm = h / image_height


            label_lines.append(f"0 {x_center:.6f} {y_center:.6f} {w_norm:.6f} {h_norm:.6f}")


        txt_path = os.path.join(abs_output_dir, f"V{setnum:03d}_{vbbnum:05d}.txt")
        with open(txt_path, 'w') as f:
            f.write('\n'.join(label_lines))




    print(f"Done. {n_frames} label files saved in:\n{abs_output_dir}")

imgnumtxt = 0
NUMBER_TRAIN_SETS = 6
NUMBER_TEST_SETS = 5
for s in range(NUMBER_TRAIN_SETS):
    vbb_set_dir = os.path.join(data_folder_dir, f'annotations/annotations/set{s:02d}')
    vbb_files = os.listdir(vbb_set_dir) # number of vbb files in each set
    NUMBER_OF_VB = len(vbb_files)
    #print(NUMBER_OF_VB)
    for v in range(NUMBER_OF_VB):
        # dont need to change this dir
        vbb_path = os.path.join(vbb_set_dir, f'V{v:03d}.vbb') 
        convert_vbb_to_yolo(vbb_path, converted_labels_train_dir, s, v, image_width=640, image_height=480)

for s in range(NUMBER_TRAIN_SETS+1, NUMBER_TRAIN_SETS + NUMBER_TEST_SETS):
    vbb_set_dir = os.path.join(data_folder_dir, f'annotations/annotations/set{s:02d}')
    vbb_files = os.listdir(vbb_set_dir) # number of vbb files in each set
    NUMBER_OF_VB = len(vbb_files)
    #print(NUMBER_OF_VB)
    for v in range(NUMBER_OF_VB):
        # dont need to change this dir
        vbb_path = os.path.join(vbb_set_dir, f'V{v:03d}.vbb') 
        convert_vbb_to_yolo(vbb_path, converted_labels_test_dir, s, v, image_width=640, image_height=480)



Reading .vbb file: C:/Users/dasul/Downloads/data_and_labels\annotations/annotations/set00\V000.vbb
Done. 1845 label files saved in:
C:\Users\dasul\Downloads\data_and_labels\processed_data\labels\train
Reading .vbb file: C:/Users/dasul/Downloads/data_and_labels\annotations/annotations/set00\V001.vbb
Done. 1844 label files saved in:
C:\Users\dasul\Downloads\data_and_labels\processed_data\labels\train
Reading .vbb file: C:/Users/dasul/Downloads/data_and_labels\annotations/annotations/set00\V002.vbb
Done. 1322 label files saved in:
C:\Users\dasul\Downloads\data_and_labels\processed_data\labels\train
Reading .vbb file: C:/Users/dasul/Downloads/data_and_labels\annotations/annotations/set00\V003.vbb
Done. 568 label files saved in:
C:\Users\dasul\Downloads\data_and_labels\processed_data\labels\train
Reading .vbb file: C:/Users/dasul/Downloads/data_and_labels\annotations/annotations/set00\V004.vbb
Done. 2133 label files saved in:
C:\Users\dasul\Downloads\data_and_labels\processed_data\labels\tr

Create the YAML file

In [27]:
import yaml
data = {
    'train': converted_images_train_dir,
    'val': converted_images_test_dir,
    'nc': 1,
    'names': ['pedestrian']
}

yaml_dir = os.path.join(converted_data_dir, 'data.yaml')
with open(yaml_dir, 'w') as file:
    yaml.dump(data, file, default_flow_style=False)

print("data.yaml created successfully!")

data.yaml created successfully!


# Train the model

idk why this was so buggy, I had to inlcude the absolute path for the model becuase otherwise it would just not recognise the yaml file
another error i got was in the line: img = torch.tensor(img).float() / 255.0, and i got a value error, becaus ethe numpy array had negative strides, and we were trying to convert into a pytorch tensor, so i replaced the line with:
img = torch.tensor(img.copy()).float() / 255.0, which was an edit to the custom_yolo_dataset file

old

In [10]:
# train.py
# Custom training script for YOLOv5 pedestrian detection project

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import sys
import os

sys.path.append(os.path.join(os.getcwd(), 'yolov5'))
from yolov5.models.yolo import Model

sys.path.append(os.path.join(os.getcwd(), 'custom_yolo_dataset'))
from custom_yolo_dataset import YOLODataset

# Device setup 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the YOLOv5 small model architecture (1 class = person)
# idk but this part wasnt working as it was so I used the absolute path
cfg_path = "C:/Users/dasul/156/yolov5/models/yolov5s.yaml"
model = Model(cfg=cfg_path, ch=3, nc=1).to(device)

# Initialize weights for better training
def init_weights(m):
    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
        if m.bias is not None:
            nn.init.zeros_(m.bias)

model.apply(init_weights)

# Set image resolution to higher value for small object detection (e.g. pedestrians)
img_size = 640

# Load training data using our custom dataset class
train_dataset = YOLODataset(
    images_dir= converted_images_train_dir,
    labels_dir= converted_labels_train_dir,
    img_size=img_size
)

## had to add this functino as well to deal with the fact that we were getting tensors of different size, and added it to trainloader and targets
def yolo_collate_fn(batch):
    images, labels = zip(*batch)
    images = torch.stack(images, dim=0)
    return images, labels

# Wrap the dataset in a DataLoader
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=yolo_collate_fn)

# Set up optimizer and learning rate scheduler (with decay)
optimizer = optim.SGD(model.parameters(), lr=0.005, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.5)  # LR halves every 3 epochs

# Simple placeholder loss 
def yolo_loss(pred, targets):
    return pred[0].sum() * 0.0  # dummy loss, for structure only

# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    total_loss = 0

    for imgs, targets in train_loader:
        imgs = imgs.to(device)
        targets = [t.to(device) for t in targets] # also had to make an edit here

        preds = model(imgs)
        print("here")
        loss = yolo_loss(preds, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    
    scheduler.step()  # apply learning rate decay
    print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}")
    print(f"completed epoch {epoch+1}/{epoch}")

# Save the trained model weights
torch.save(model.state_dict(), 'best.pt')
print("Training complete. Weights saved as best.pt")

Overriding model.yaml nc=80 with nc=1

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  yolov5.models.common.Conv               [3, 32, 6, 2, 2]              
  1                -1  1     18560  yolov5.models.common.Conv               [32, 64, 3, 2]                


  2                -1  1     18816  yolov5.models.common.C3                 [64, 64, 1]                   
  3                -1  1     73984  yolov5.models.common.Conv               [64, 128, 3, 2]               
  4                -1  2    115712  yolov5.models.common.C3                 [128, 128, 2]                 
  5                -1  1    295424  yolov5.models.common.Conv               [128, 256, 3, 2]              
  6                -1  3    625152  yolov5.models.common.C3                 [256, 256, 3]                 
  7                -1  1   1180672  yolov5.models.common.Conv               [256, 512, 3, 2]              
  8                -1  1   1182720  yolov5.models.common.C3                 [512, 512, 1]                 
  9                -1  1    656896  yolov5.models.common.SPPF               [512, 512, 5]                 
 10                -1  1    131584  yolov5.models.common.Conv               [512, 256, 1, 1]              
 11                -1  1         0  t

here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here
here


KeyboardInterrupt: 

# Analysing the Data

some additional changes ive made:
so for some reason you need to include the absolute path to the model otherwise it wont detect so this is gonna be speicifc to whoever is running the program
also had the same problme with converting numpy array to pytorch, this time making a copy of images was enoguh ie images.copy() instead of just converting images
Also edited some of the file location and structure

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# so now I need to create a an array of the predicted values vs the target values
# well the target values are held in 

# infer.py
# Custom inference script with annotation saving + precision/recall evaluation

import os
import cv2
import torch
import numpy as np
import sys
sys.path.append('C:/Users/dasul/156/yolov5')
#sys.path.append(os.path.join(os.getcwd(), 'yolov5'))
from models.yolo import Model
from yolov5.utils.general import non_max_suppression, scale_boxes
from yolov5.utils.augmentations import letterbox
#from yolov5.utils.plots import plot_one_box

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load YOLOv5 model architecture and trained weights
cfg_path = 'C:/Users/dasul/156/yolov5/models/yolov5s.yaml'
model = Model(cfg=cfg_path, ch=3, nc=1).to(device)
model.load_state_dict(torch.load('best.pt', map_location=device))
model.eval()

# Input and output paths
image_dir = converted_images_test_dir
gt_label_dir = converted_labels_test_dir
results_dir = os.path.join(data_folder_dir, 'results')
os.makedirs(results_dir, exist_ok=True)
output_img_dir = os.path.join(results_dir, 'inference_output/images')
output_txt_dir = os.path.join(results_dir, 'inference_output/labels')
os.makedirs(output_img_dir, exist_ok=True)
os.makedirs(output_txt_dir, exist_ok=True)

# Inference settings
img_size = 640
conf_thres = 0.5
iou_thres = 0.45

# Initialize counters
tp = 0
fp = 0
fn = 0

def plot_one_box(xyxy, img, color=(0, 255, 0), label=None, line_thickness=2):
    """
    Draws a bounding box with optional label on an image.

    Args:
        xyxy (list): [x1, y1, x2, y2]
        img (ndarray): Image to draw on
        color (tuple): BGR box color
        label (str): Optional label text
        line_thickness (int): Box line thickness
    """
    x1, y1, x2, y2 = map(int, xyxy)
    cv2.rectangle(img, (x1, y1), (x2, y2), color, thickness=line_thickness, lineType=cv2.LINE_AA)

    if label:
        font_scale = 0.5
        font_thickness = 1
        t_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, font_scale, font_thickness)[0]
        c2 = x1 + t_size[0], y1 - t_size[1] - 3
        cv2.rectangle(img, (x1, y1), c2, color, -1, cv2.LINE_AA)
        cv2.putText(img, label, (x1, y1 - 2), cv2.FONT_HERSHEY_SIMPLEX,
                    font_scale, (255, 255, 255), font_thickness, lineType=cv2.LINE_AA)

# Helper functions
def compute_iou(box1, box2):
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    inter_area = max(0, x2 - x1) * max(0, y2 - y1)
    if inter_area == 0:
        return 0.0
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    return inter_area / (box1_area + box2_area - inter_area)

def load_ground_truth(txt_path, img_shape):
    h, w = img_shape
    boxes = []
    if not os.path.exists(txt_path):
        return boxes
    with open(txt_path, 'r') as f:
        for line in f:
            cls, x, y, bw, bh = map(float, line.strip().split())
            x1 = (x - bw / 2) * w
            y1 = (y - bh / 2) * h
            x2 = (x + bw / 2) * w
            y2 = (y + bh / 2) * h
            boxes.append([x1, y1, x2, y2])
    return boxes

# Inference loop
for filename in os.listdir(image_dir):
    if not filename.endswith('.jpg'):
        continue

    img_path = os.path.join(image_dir, filename)
    img0 = cv2.imread(img_path)
    h0, w0 = img0.shape[:2]
    img = letterbox(img0, new_shape=img_size)[0]
    img = img[:, :, ::-1].transpose(2, 0, 1)
    img = torch.from_numpy(img.copy()).float() / 255.0
    img = img.unsqueeze(0).to(device)

    with torch.no_grad():
        pred = model(img)
        pred = non_max_suppression(pred, conf_thres=conf_thres, iou_thres=iou_thres)

    label_lines = []
    matched = set()
    base_name = os.path.splitext(filename)[0]
    gt_boxes = load_ground_truth(os.path.join(gt_label_dir, base_name + ".txt"), (h0, w0))

    for det in pred:
        if det is not None and len(det):
            det[:, :4] = scale_boxes(img.shape[2:], det[:, :4], img0.shape).round()
            for *xyxy, conf, cls in det:
                x1, y1, x2, y2 = [coord.item() for coord in xyxy]
                plot_one_box([x1, y1, x2, y2], img0, label=f"person {conf:.2f}")

                # YOLO-format normalized box
                x_center = ((x1 + x2) / 2) / w0
                y_center = ((y1 + y2) / 2) / h0
                bw = (x2 - x1) / w0
                bh = (y2 - y1) / h0
                label_lines.append(f"0 {x_center:.6f} {y_center:.6f} {bw:.6f} {bh:.6f}") 

                # Compare with ground truth for precision
                found_match = False
                for i, gt in enumerate(gt_boxes):
                    iou = compute_iou([x1, y1, x2, y2], gt)
                    if iou > 0.5 and i not in matched:
                        tp += 1
                        matched.add(i)
                        found_match = True
                        break
                if not found_match:
                    fp += 1
        else:
            if len(gt_boxes) > 0:
                fn += len(gt_boxes)

    # Save annotated image
    cv2.imwrite(os.path.join(output_img_dir, filename), img0)
    print("saved")

    # Save YOLO label txt
    with open(os.path.join(output_txt_dir, base_name + ".txt"), 'w') as f:
        f.write('\n'.join(label_lines))

# Compute and print precision/recall
precision = tp / (tp + fp) if (tp + fp) > 0 else 0
recall = tp / (tp + fn) if (tp + fn) > 0 else 0

print(f"\n✅ Inference complete. Results saved to 'inference_output/'")
print(f"🔍 Precision: {precision:.4f}")
print(f"📦 Recall:    {recall:.4f}")
print(f"TP: {tp}, FP: {fp}, FN: {fn}")

C:\Users/dasul/156/yolov5\utils\general.py:32: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources as pkg
Overriding model.yaml nc=80 with nc=1

                 from  n    params  module                                  arguments                     
  0                -1  1      3520  models.common.Conv                      [3, 32, 6, 2, 2]              
  1                -1  1     18560  models.common.Conv                      [32, 64, 3, 2]                
  2                -1  1     18816  models.common.C3                        [64, 64, 1]                   
  3                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  4                -1  2    115712  models.common.C3                        [128, 128, 2]                 


saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
saved
save